In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

In [ ]:
spark = SparkSession.builder.appName("SparkJoins").getOrCreate()


In [ ]:
## Create data frame 1
d1 = [
    (1,"Alice"),
    (2,"Bob"),
    (3,"Charlie"),
    (4,"David"),
    (5,"Eve")
]
df1 = spark.createDataFrame(d1,["id","name"])

## Create data frame 2
d2 = [
    (1,50000),
    (2,60000),
    (3,70000),
    (4,80000),
    (5,90000)
]
df2 = spark.createDataFrame(d2,["id","salary"])

In [ ]:
df_join_board=df1.join(broadcast(df2),df1['id']==df2['id'],"left")

In [ ]:
df_join_board.show()

+---+-------+---+------+
| id|   name| id|salary|
+---+-------+---+------+
|  1|  Alice|  1| 50000|
|  2|    Bob|  2| 60000|
|  3|Charlie|  3| 70000|
|  4|  David|  4| 80000|
|  5|    Eve|  5| 90000|
+---+-------+---+------+



In [ ]:
df_join_board.explain(True)

== Parsed Logical Plan ==
Join LeftOuter, (id#158L = id#162L)
:- LogicalRDD [id#158L, name#159], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [id#162L, salary#163L], false

== Analyzed Logical Plan ==
id: bigint, name: string, id: bigint, salary: bigint
Join LeftOuter, (id#158L = id#162L)
:- LogicalRDD [id#158L, name#159], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [id#162L, salary#163L], false

== Optimized Logical Plan ==
Join LeftOuter, (id#158L = id#162L), rightHint=(strategy=broadcast)
:- LogicalRDD [id#158L, name#159], false
+- Filter isnotnull(id#162L)
   +- LogicalRDD [id#162L, salary#163L], false

== Physical Plan ==
*(2) BroadcastHashJoin [id#158L], [id#162L], LeftOuter, BuildRight, false
:- *(2) Scan ExistingRDD[id#158L,name#159]
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=546]
   +- *(1) Filter isnotnull(id#162L)
      +- *(1) Scan ExistingRDD[id#162L,salary#163L]

